## Resolvendo o Ambiente do Taxi com Algoritmo Q-Learning

In [ ]:
# openai.com gym

In [ ]:
#!pip install gym

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make("Taxi-v2")
state = env.reset()

/Users/vladimiralencar/anaconda3/lib/python3.7/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
print(state)
env.render()

367
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



<h1>Ações Possíveis</h1>

down (0), up (1), right (2), left (3), pick-up (4), and drop-off (5)

In [4]:
n_states = env.observation_space.n
n_actions = env.action_space.n

In [5]:
n_states, n_actions

(500, 6)

<h1>Quão bom se comporta de forma completamente aleatória?</h1>

In [6]:
state = env.reset()
counter = 0
g = 0
reward = None
while reward != 20:
    state, reward, done, info = env.step(env.action_space.sample())
    counter += 1
    g += reward
print("Resolvido em {} passos com uma recompensa total de {}".format(counter,g))

Resolvido em 3142 passos com uma recompensa total de -12229


## Vamos analisar apenas um episódio e ver como os valores de Q mudam após cada etapa usando a fórmula abaixo

In [9]:
Q = np.zeros([n_states, n_actions])

# Esta matriz multidimensional manterá um histórico de nossos Q-Values para todos os estados
Q_hist = np.zeros([n_states, n_actions, 0])


episodes = 1
G = 0
alpha = 0.618
counter = 0
for episode in range(1,episodes+1):
    done = False
    G, reward = 0,0
    state = env.reset()
    firstState = state
    print("Initial State = {}".format(state))
    while reward != 20:
        action = np.argmax(Q[state]) 
        state2, reward, done, info = env.step(action)
        Q[state,action] += alpha * (reward + np.max(Q[state2]) - Q[state,action]) 
        G += reward
        state = state2
        counter += 1
        # Isso manterá um histórico de Q Values em uma matriz multidimensional
        Q_hist = np.dstack((Q_hist, Q))
finalState = state
print("Final State = {}".format(finalState))


Initial State = 112
Final State = 16


In [10]:
episodes, reward, counter

(1, 20, 673)

<img src="images/qlearn.png">

<h1>Vejamos o primeiro passo:<h1>

In [11]:
print(firstState)
Q_hist[firstState,:,0]

112


array([-0.618,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ])

<h1>Vejamos o passo final:</h1>

In [12]:
Q_hist.shape

(500, 6, 673)

In [13]:
print(finalState)
Q_hist[finalState,:,Q_hist.shape[2]-1]

16


array([-0.618, -0.618, -0.618, -0.618, -6.18 , 12.36 ])

<h1>Você espera que a Q-Table tenha um bom valor para o segundo último passo?<h1>

In [14]:
np.argmax(Q_hist[:,:,Q_hist.shape[2]-2], axis = 0)

array([0, 0, 0, 0, 0, 0])

<h1>Não, mas definitivamente sabe quais ações seriam as piores<h1>

In [15]:
np.argmin(Q_hist[:,:,Q_hist.shape[2]-2], axis = 0)

array([412, 412, 112,  12,  12,  12])

<h1>Vamos passar por vários episódios para que possamos convergir para uma política ótima</h1>

In [16]:
episodes = 2000
rewardTracker = []

G = 0
alpha = 0.618

for episode in range(1,episodes+1):
    done = False
    G, reward = 0,0
    state = env.reset()
    while done != True:
        action = np.argmax(Q[state]) 
        state2, reward, done, info = env.step(action) 
        Q[state,action] += alpha * ((reward + (np.max(Q[state2]))  - Q[state,action]))
        G += reward
        state = state2
    
    if episode % 100 == 0:
        print('Episode {} Total Reward: {}'.format(episode,G))

Episode 100 Total Reward: -117
Episode 200 Total Reward: -35
Episode 300 Total Reward: 13
Episode 400 Total Reward: 1
Episode 500 Total Reward: 12
Episode 600 Total Reward: 11
Episode 700 Total Reward: 9
Episode 800 Total Reward: 10
Episode 900 Total Reward: 8
Episode 1000 Total Reward: 11
Episode 1100 Total Reward: 5
Episode 1200 Total Reward: 8
Episode 1300 Total Reward: 8
Episode 1400 Total Reward: 11
Episode 1500 Total Reward: 11
Episode 1600 Total Reward: 9
Episode 1700 Total Reward: 9
Episode 1800 Total Reward: 9
Episode 1900 Total Reward: 10
Episode 2000 Total Reward: 7


<h1>Agora que aprendemos os Q values ideais, desenvolvemos uma política ótima e não precisamos mais treinar o agente.<h>

In [17]:
state = env.reset()
done = None

while done != True:
    # Simplesmente tomamos a ação com o valor Q mais alto
    action = np.argmax(Q[state])
    state, reward, done, info = env.step(action)
    env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : :_: : |
| | : | : |
|Y